In [27]:
import pandas as pd

In [28]:
df = pd.read_json('results.jsonl', lines=True)

In [29]:
df.head()

,model_id,similarity_function,problematic_queries_count,problematic_queries,avg_embedding_time,accuracy@k,precision@k,recall@k,ndcg@k,mrr@k,map@k
0,Alibaba-NLP/gte-large-en-v1.5,cosine,4,"[""OneZero digital digest"", ""Photo ID pickup de...",0.126968,"{'1': 0.6590909091, '3': 0.8863636364, '5': 0....","{'1': 0.6590909091, '3': 0.321969697, '5': 0.2...","{'1': 0.556344697, '3': 0.7637310606000001, '5...",{'10': 0.1962689817},{'10': 0.7725649351},{'100': 0.6825948938}
1,iamgroot42/spice,cosine,12,"[""meeting with Domingo Guerra after June 19, 2...",0.000000,"{'1': 0.625, '3': 0.7727272727000001, '5': 0.8...","{'1': 0.625, '3': 0.2878787879, '5': 0.1863636...","{'1': 0.5440340909, '3': 0.7059659091, '5': 0....",{'10': 0.1806189877},{'10': 0.7067685786},{'100': 0.6546213018}
2,intfloat/e5-large-v2,cosine,1,"[""energy efficiency report comparison October ...",0.000000,"{'1': 0.7840909091, '3': 0.8863636364, '5': 0....","{'1': 0.7840909091, '3': 0.321969697, '5': 0.2...","{'1': 0.6766098485, '3': 0.787405303, '5': 0.8...",{'10': 0.2127621926},{'10': 0.8456890332},{'100': 0.7692223207000001}
3,ibm-granite/granite-embedding-125m-english,cosine,4,"[""GLS 508912991 delivery notice"", ""ASP.NET Cor...",0.000000,"{'1': 0.8181818182, '3': 0.8977272727000001, '...","{'1': 0.8181818182, '3': 0.33333333330000003, ...","{'1': 0.6889204545000001, '3': 0.7883522727000...",{'10': 0.2165859235},{'10': 0.8621212121},{'100': 0.7741798112}
4,sentence-transformers/all-MiniLM-L6-v2,cosine,11,"[""Zoom invitation on April 21, 2021"", ""Pat ask...",0.000000,"{'1': 0.6477272727000001, '3': 0.7159090909, '...","{'1': 0.6477272727000001, '3': 0.2575757576, '...","{'1': 0.5667613636000001, '3': 0.6439393939, '...",{'10': 0.1773332362},{'10': 0.7061056999},{'100': 0.6501311296}


In [30]:
# format the output

flattened_data = {}
score_columns = ["accuracy@k", "precision@k", "recall@k", "ndcg@k", "mrr@k", "map@k"]
for col in score_columns:
    for key in df[col][0].keys():
        flattened_data[f"{col}_{key}"] = df[col].apply(lambda x: f"{x[key] * 100:.2f}%" if key in x else "0%")

flattened_df = pd.DataFrame(flattened_data)
df = pd.concat([df.drop(columns=score_columns), flattened_df], axis=1)

In [31]:
cols = list(df.columns)
sorted_metrics = [col for col in cols if any(metric in col for metric in score_columns)]
new_order = ["model_id"] + sorted_metrics + [col for col in cols if col not in sorted_metrics and col != "model_id"]
df = df[new_order]

In [32]:
df

,model_id,accuracy@k_1,accuracy@k_3,accuracy@k_5,accuracy@k_10,precision@k_1,precision@k_3,precision@k_5,precision@k_10,recall@k_1,recall@k_3,recall@k_5,recall@k_10,ndcg@k_10,mrr@k_10,map@k_100,similarity_function,problematic_queries_count,problematic_queries,avg_embedding_time
0,Alibaba-NLP/gte-large-en-v1.5,65.91%,88.64%,89.77%,95.45%,65.91%,32.20%,20.23%,11.14%,55.63%,76.37%,78.36%,82.95%,19.63%,77.26%,68.26%,cosine,4,"[""OneZero digital digest"", ""Photo ID pickup de...",0.126968
1,iamgroot42/spice,62.50%,77.27%,82.95%,86.36%,62.50%,28.79%,18.64%,10.23%,54.40%,70.60%,74.72%,78.55%,18.06%,70.68%,65.46%,cosine,12,"[""meeting with Domingo Guerra after June 19, 2...",0.000000
2,intfloat/e5-large-v2,78.41%,88.64%,92.05%,98.86%,78.41%,32.20%,20.91%,11.82%,67.66%,78.74%,82.67%,88.54%,21.28%,84.57%,76.92%,cosine,1,"[""energy efficiency report comparison October ...",0.000000
3,ibm-granite/granite-embedding-125m-english,81.82%,89.77%,93.18%,95.45%,81.82%,33.33%,21.36%,11.70%,68.89%,78.84%,81.68%,86.27%,21.66%,86.21%,77.42%,cosine,4,"[""GLS 508912991 delivery notice"", ""ASP.NET Cor...",0.000000
4,sentence-transformers/all-MiniLM-L6-v2,64.77%,71.59%,80.68%,87.50%,64.77%,25.76%,17.27%,10.11%,56.68%,64.39%,72.92%,80.49%,17.73%,70.61%,65.01%,cosine,11,"[""Zoom invitation on April 21, 2021"", ""Pat ask...",0.000000
5,sentence-transformers/multi-qa-mpnet-base-dot-v1,57.95%,79.55%,86.36%,90.91%,57.95%,28.79%,18.64%,10.34%,50.14%,70.50%,76.28%,81.49%,17.82%,69.84%,63.36%,cosine,8,"[""ASP.NET Core features email"", ""Zoom invitati...",0.000000
6,thenlper/gte-base,75.00%,86.36%,92.05%,96.59%,75.00%,31.82%,20.68%,11.14%,63.97%,75.80%,79.92%,83.71%,20.39%,81.94%,73.14%,cosine,3,"[""Photo ID pickup details for Igor Rendulic"", ...",0.000000
7,intfloat/e5-small-v2,71.59%,82.95%,92.05%,96.59%,71.59%,30.68%,20.68%,11.25%,59.42%,73.91%,81.25%,85.84%,20.07%,79.55%,70.82%,cosine,3,"[""Akontacija DDPO junij 2020"", ""Photo ID picku...",0.000000
